# Intermediate Machine Learning Micro 

An expansion on the concepts and methods used in the intro to machine learning micro course

## Missing Values

There are 3 widely accepted strategies fro dealing with missing values

1. Drop the column that contains missing values.
    - This approach generally leads to less accurate models since we are throwing out a lot of potentialls helpful information.

2. Replace the missing values with a mean from that column (imputation).
    - This generally leads to more accurate models but still can cause a skew / innacuray.

3. Extension to Imputation:
    - this approach is a combination and extension of approach #2 where an extra column is added that indicated whether or not that column contained a missing value to begin with.

In [27]:
# Import data and set up training / validation sets
import pandas as pd
from sklearn.model_selection import train_test_split

MELBOURNE_DATA_PATH = '..\\data\\melb_data.csv'

melb_data = pd.read_csv(MELBOURNE_DATA_PATH)
melb_predictors = melb_data.drop(['Price'], axis=1)

y = melb_data.Price
X = melb_predictors.select_dtypes(exclude=['object'])

X_train, X_val, y_train, y_val = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)


In [28]:
# Define funciton to return MAE score for the data
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor

def score_dataset(X_train, X_val, y_train, y_val) -> float:
    model = RandomForestRegressor(n_estimators=10, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_val)
    return mean_absolute_error(y_val, preds)

First we will try approach #1 to see what our score is. 

In [29]:
# Get the names of columns with missing values
cols_missing_vals = [col for col in X_train.columns if X_train[col].isnull().any()]

# Drop columns in the training and validation data that are missing values
dropna_X_train = X_train.drop(cols_missing_vals, axis=1)
dropna_X_val = X_val.drop(cols_missing_vals, axis=1)

print(f'MAE for approach 1: {score_dataset(dropna_X_train, dropna_X_val, y_train, y_val)}')

MAE for approach 1: 183550.22137772635


Next lets see how imputation (strategy #2) fairs compared to approach #1

In [30]:
# First we need the mean values of the columns missing values
cols_missing_vals_mean = [X_train[column].mean() for column in cols_missing_vals]

imputed_X_train = X_train.copy()
imputed_X_val = X_val.copy()

for i, column in enumerate(cols_missing_vals):
    imputed_X_train[column] = X_train[column].fillna(cols_missing_vals_mean[i])
    imputed_X_val[column] = X_val[column].fillna(cols_missing_vals_mean[i])

# Custom solution! but theres a faster way to do it...
print(f'MAE for approach 2 (Imputation): {score_dataset(imputed_X_train, imputed_X_val, y_train, y_val)}')


from sklearn.impute import SimpleImputer

# Imputation
my_imputer = SimpleImputer()
imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))
imputed_X_valid = pd.DataFrame(my_imputer.transform(X_val))

# Imputation removed column names; put them back
imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_val.columns

print(f'MAE from Approach 2 (Imputation): {score_dataset(imputed_X_train, imputed_X_valid, y_train, y_val)}')

MAE for approach 2 (Imputation): 178166.46269899711
MAE from Approach 2 (Imputation): 178166.46269899711


Now lets try method #3 imputation + extension.

In [31]:
# First copy the training and validation data to new variables
X_train_plus = X_train.copy()
X_val_plus = X_val.copy()

# Make a new column for each column that had missing values
for col in cols_missing_vals:
    X_train_plus[col + '_was_missing'] = X_train_plus[col].isnull()
    X_val_plus[col + '_was_missing'] = X_val_plus[col].isnull()

# Imputation
my_imputer = SimpleImputer()
imputed_X_train_plus = pd.DataFrame(my_imputer.fit_transform(X_train_plus))
imputed_X_val_plus = pd.DataFrame(my_imputer.transform(X_val_plus))

# Imputation removed column names; put them back
imputed_X_train_plus.columns = X_train_plus.columns
imputed_X_val_plus.columns = X_val_plus.columns

print(f'MAE from approach 3 (Imputation + Extension): {score_dataset(imputed_X_train_plus, imputed_X_val_plus, y_train, y_val)}')


MAE from approach 3 (Imputation + Extension): 178927.503183954


One of the reasons dropping the columns is not a good strategy especially the larger the data set is the sheer number of data points that is lost:

In [32]:
# Shape of training data (num_rows, num_columns)
print(X_train.shape)

# Number of missing values in each column of training data
missing_val_count_by_column = (X_train.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

(10864, 12)
Car               49
BuildingArea    5156
YearBuilt       4307
dtype: int64
